In [5]:
import sys
import csv
from selenium import webdriver
import time

# default path to file to store data
path_to_file = "reviews.csv"

# default number of scraped pages
num_page = 10

# default tripadvisor website of restaurant
url = "https://www.tripadvisor.com/Restaurant_Review-g60763-d802686-Reviews-Hard_Rock_Cafe-New_York_City_New_York.html"

# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

# Import the webdriver
driver = webdriver.Proxy()
driver.get(url)

# Open the file to save the review
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

# change the value inside the range to save more or less reviews
for i in range(0, num_page):
    
    # expand the review 
    time.sleep(2)
    driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()

    container = driver.find_elements_by_xpath(".//div[@class='review-container']")

    for j in range(len(container)):

        title = container[j].find_element_by_xpath(".//span[@class='noQuotes']").text
        date = container[j].find_element_by_xpath(".//span[contains(@class, 'ratingDate')]").get_attribute("title")
        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        review = container[j].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", " ")

        csvWriter.writerow([date, rating, title, review]) 

    # change the page
    driver.find_element_by_xpath('.//a[@class="nav next ui_button primary"]').click()

driver.close()

AttributeError: 'Proxy' object has no attribute 'get'

In [ ]:
import time
import math
import re
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
 
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os
import logging
import shutil
import time
from pathlib import Path
import undetected_chromedriver as uc
 


class TripadvisorScraper:
    def __init__(self, url):
        self.url = url
        self.nom_restaurant = None
        self.nb_total_commentaires = None
        self.nb_pages = None
        self.nb_commentaires_par_page = None
        self.data = None
        
        self.driver = None

    def create_driver(self):
        service = Service('chromedriver')

        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        ]
        options = uc.ChromeOptions()
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument("--incognito")
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        user_agent = random.choice(user_agents)
        options.add_argument(f'--user-agent={user_agent}')
        return uc.Chrome(options=options, service=service)        
        # browser_executable_path = shutil.which("chromium")

        # Path('selenium.log').unlink(missing_ok=True)
        # time.sleep(1)

        # options = uc.ChromeOptions()
        # user_agents = [
        # 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        # ]
        # options.add_argument("--headless")
        # options.add_argument("--no-sandbox")
        # options.add_argument("--disable-dev-shm-usage")
        # options.add_argument("--disable-gpu")
        # options.add_argument("--disable-features=NetworkService")
        # options.add_argument("--window-size=1920x1080")
        # options.add_argument("--disable-features=VizDisplayCompositor")
        # user_agent = random.choice(user_agents)
        # options.add_argument(f'--user-agent={user_agent}')

        # return uc.Chrome(browser_executable_path=browser_executable_path,
        #         options=options,
        #         use_subprocess=False,
        #         log_level=logging.DEBUG,
        #         service_log_path='selenium.log')
    
    def handle_cookies(self):
        try:
            WebDriverWait(self.driver, 5).until(
            # WebDriverWait(self.driver, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-accept-btn-handler']"))
            ).click()
        except TimeoutException:
            print("Pas de bannière cookies trouvée.")

    def find_restaurant_name(self):
        try:
            name_element = self.driver.find_element(By.XPATH, "//h1[@class='biGQs _P egaXP rRtyp']")
            print(f"Nom trouvé : {name_element.text}")
            self.nom_restaurant = name_element.text
            return name_element.text
        except NoSuchElementException:
            return None

    def extraire_infos(self, texte):
        texte = texte.replace("\u202f", "")
        chiffres = [int(s) for s in re.findall(r'\d+', texte)]
        
        if len(chiffres) >= 2:
            nb_commentaires_par_page = chiffres[1]
            nb_total_commentaires = chiffres[-1]
            nb_pages = math.ceil(nb_total_commentaires / nb_commentaires_par_page)
            self.nb_total_commentaires = nb_total_commentaires
            self.nb_pages = nb_pages
            self.nb_commentaires_par_page = nb_commentaires_par_page
            # return nb_commentaires_par_page, nb_total_commentaires, nb_pages
            return nb_commentaires_par_page, nb_total_commentaires, 2
        else:
            return None, None, None

    def scraper_infos_restaurant(self):
        try:
            nom = self.driver.find_element(By.XPATH, "//h1[@class='biGQs _P egaXP rRtyp']").text
            adresse = self.driver.find_element(By.XPATH, "//div[contains(text(), 'Emplacement et coordonnées')]/following::span[contains(@class, 'biGQs _P pZUbB hmDzD')][1]").text
            note_globale = re.search(r"(\d+,\d+)", self.driver.find_elements(By.XPATH, "//div[@class='biGQs _P vvmrG']")[0].text).group(1)
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span//button[@class='ypcsE _S wSSLS']"))).click()
            horaires = [
                f"{lines[0]} : {' - '.join(lines[1:])}"
                for e in self.driver.find_elements("xpath", "//div[@class='VFyGJ Pi']")
                if len(lines := e.text.splitlines()) >= 2
            ]

            time.sleep(3)
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span//button[@class='ypcsE _S wSSLS']"))).click()

            notes = self.driver.find_elements(By.XPATH, "//div[@class='khxWm f e Q3']/div/div")
            note_cuisine = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[1].get_attribute("innerHTML")).group(1)
            note_service = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[3].get_attribute("innerHTML")).group(1)
            note_rapportqualiteprix = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[5].get_attribute("innerHTML")).group(1)
            note_ambiance = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[7].get_attribute("innerHTML")).group(1)
            classement_element = self.driver.find_element(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB hmDzD')]//b/span").text.strip()
            classement = (re.search(r'\d+', classement_element).group())

            WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[class='UikNM _G B- _S _W _T c G_ wSSLS ACvVd']"))).click()
            time.sleep(2)

            try:
                infos_pratiques = self.driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'Infos pratiques')]]/following-sibling::div[contains(@class, 'biGQs')]").text.strip()
            except Exception:
                infos_pratiques = "Non renseigné"

            try:
                fourchette_prix = self.driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'FOURCHETTE DE PRIX')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD')]").text.strip().replace("€", "").replace("\xa0", "")
            except Exception:
                fourchette_prix = "Non renseigné"

            try:
                types_cuisines = [item.strip() for item in self.driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'CUISINES')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD')]").text.strip().split(",")]
            except Exception:
                types_cuisines = "Non renseigné"

            try:
                regimes = [item.strip() for item in self.driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'Régimes spéciaux')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD')]").text.strip().split(",")]
            except Exception:
                regimes = "Non renseigné"

            try:
                repas = [item.strip() for item in self.driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'Repas')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW eWlDX GzNcM ATzgx UTQMg TwpTY hmDzD')]").text.strip().split(",")]
            except Exception:
                repas = "Non renseigné"

            try:
                fonctionnalites = [item.strip() for item in self.driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'FONCTIONNALITÉS')]]/following-sibling::div[contains(@class, 'biGQs')]").text.strip().split(",")]
            except Exception:
                fonctionnalites = "Non renseigné"

            time.sleep(5)
            self.driver.find_element(By.XPATH, "//button[@aria-label='Fermer']").click()
            time.sleep(2)

            try:
                google_maps_link = self.driver.find_element(By.XPATH,"//div[@class='akmhy e j']//a[@class='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS']").get_attribute("href")
                if "@" in google_maps_link:
                    coordinates = google_maps_link.split("@")[1].split(",")[:2]
                    latitude, longitude = coordinates[0], coordinates[1]
                else:
                    latitude, longitude = "Non renseigné", "Non renseigné"
                    print("Coordonnées introuvables dans le lien.")
            except NoSuchElementException:
                latitude, longitude = "Non renseigné", "Non renseigné"
                print("Lien Google Maps introuvable.")

            return {
                "nom": nom,
                "adresse": adresse,
                "classement": classement,
                "horaires": horaires,
                "note_globale": note_globale,
                "note_cuisine": note_cuisine,
                "note_service": note_service,
                "note_rapportqualiteprix": note_rapportqualiteprix,
                "note_ambiance": note_ambiance,
                "infos_pratiques": infos_pratiques,
                "repas": repas,
                "regimes": regimes,
                "fonctionnalites": fonctionnalites,
                "fourchette_prix": fourchette_prix,
                "types_cuisines": types_cuisines,
                "latitude": latitude,
                "longitude": longitude
            }
        except Exception as e:
            print(f"Erreur lors de l'extraction des informations du restaurant : {e}")
            return {}

    def scraper_page(self):
        data = []
        pseudos = self.driver.find_elements(By.XPATH, "//span[@class='biGQs _P fiohW fOtGX']")
        titres = self.driver.find_elements(By.XPATH, "//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']")
        etoiles = self.driver.find_elements(By.XPATH, "//div[@class='OSBmi J k']")
        nb_etoiles = [re.search(r'(\d+),', etoile.get_attribute("textContent")).group(1) for etoile in etoiles]
        dates = [re.search(r"\d{1,2}\s\w+\s\d{4}", elem.text.strip()).group(0) for elem in self.driver.find_elements(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB ncFvv osNWb')]")]
        experiences = self.driver.find_elements(By.XPATH, "//span[@class='DlAxN']")
        reviews = self.driver.find_elements(By.XPATH, "//div[@data-test-target='review-body']//span[@class='JguWG' and not(ancestor::div[contains(@class, 'csNQI')])]")

        for i in range(len(titres)):
            avis = {
                "pseudo": pseudos[i].text if i < len(pseudos) else "",
                "titre_review": titres[i].text if i < len(titres) else "",
                "nb_etoiles": nb_etoiles[i] if i < len(nb_etoiles) else "",
                "date": dates[i] if i < len(dates) else "",
                "experience": experiences[i].text if i < len(experiences) else "",
                "review": reviews[i].text if i < len(reviews) else ""
            }
            data.append(avis)
        return data

    def scraper_toutes_pages(self, nb_pages):
        all_data = []
        actions = ActionChains(self.driver)

        for page in range(1, nb_pages + 1):
            print(f"Scraping de la page {page}...")
            time.sleep(5)
            try:
                data = self.scraper_page()
                print(f"Données collectées pour la page {page} : {len(data)} avis")
                all_data.extend(data)

                next_button = WebDriverWait(self.driver, 50).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Page suivante']"))
                )

                self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(5)
                actions.move_to_element(next_button).click().perform()

                print("Page suivante chargée.")
            except Exception as e:
                print(f"Erreur rencontrée à la page {page} : {e}")
                break

        return all_data

    def test_scraping(self, nbPages_texte):
        avis = []
        infos_restaurant = {
            "nom": "Non disponible",
            "adresse": "Non disponible",
            "classement": "Non disponible",
            "horaires": [],
            "note_globale": "Non disponible",
            "note_cuisine": "Non disponible",
            "note_service": "Non disponible",
            "note_rapportqualiteprix": "Non disponible",
            "note_ambiance": "Non disponible",
            "repas": "Non disponible",
            "infos_pratiques": "Non disponible",
            "regimes": [],
            "fonctionnalites": "Non disponible",
            "fourchette_prix": "Non disponible",
            "types_cuisines": [],
            "latitude" : "Non disponible",
            "longitude" : "Non disponible",
        }
        try:
            infos_restaurant = self.scraper_infos_restaurant()
            nb_commentaires_par_page, nb_total_commentaires, nb_pages = self.extraire_infos(nbPages_texte)

            average_time_per_page = 15
            estimated_total_time = average_time_per_page * nb_pages
            estimated_total_time_minutes = math.ceil(estimated_total_time / 60)
            print(f"Temps estimé pour terminer le scraping : {estimated_total_time_minutes} minutes.\n")

            avis = self.scraper_toutes_pages(nb_pages)
            print(f"Scraping terminé. Total d'avis collectés : {len(avis)}")

        except Exception as e:
            print(f"Erreur générale : {e}")

        restaurant_data = {
            "nom": infos_restaurant["nom"],
            "adresse": infos_restaurant["adresse"],
            "classement": infos_restaurant["classement"],
            "horaires": infos_restaurant["horaires"],
            "note_globale": infos_restaurant["note_globale"],
            "note_cuisine": infos_restaurant["note_cuisine"],
            "note_service": infos_restaurant["note_service"],
            "note_rapportqualiteprix": infos_restaurant["note_rapportqualiteprix"],
            "note_ambiance": infos_restaurant["note_ambiance"],
            "infos_pratiques": infos_restaurant["infos_pratiques"],
            "repas": infos_restaurant["repas"],
            "regimes": infos_restaurant["regimes"],
            "fourchette_prix": infos_restaurant["fourchette_prix"],
            "fonctionnalités": infos_restaurant["fonctionnalites"],
            "type_cuisines": infos_restaurant["types_cuisines"],
            "latitude": infos_restaurant["latitude"],
            "longitude": infos_restaurant["longitude"],
            "avis": avis
        }

        return restaurant_data

    def scrapper(self):
        found = False
        attempts = 0
        max_attempts = 20

        while not found and attempts < max_attempts:
            self.driver = self.create_driver()
            try:
                self.driver.get(self.url)
                time.sleep(3)
                self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
                time.sleep(3)
                #save hltm page
                with open("test.html", "w") as file:
                    file.write(self.driver.page_source)

                self.handle_cookies()

                if self.find_restaurant_name():
                    found = True
            except NoSuchElementException:
                print(f"Nom non trouvé, tentative {attempts + 1}/{max_attempts}. Redémarrage...")
                attempts += 1
                self.cleanup()
                time.sleep(10)

        if not found:
            print("Échec : le nom n'a pas été trouvé après plusieurs tentatives.")
            self.cleanup()
        else:
            print("Le nom a été trouvé avec succès. Le navigateur reste ouvert.")
            nbPages_texte = self.driver.find_element("xpath", "//div[@class='Ci']").text
            data = self.test_scraping(nbPages_texte)
            self.cleanup()
            self.data = data
            # return data

    def cleanup(self):
        if self.driver:
            self.driver.quit()
            time.sleep(2)


    def __del__(self):
        self.cleanup()

    def save_data(self, data):
        pass

    
def main():
    # url = "https://www.tripadvisor.fr/Restaurant_Review-g187265-d5539701-Reviews-L_Institut_Restaurant-Lyon_Rhone_Auvergne_Rhone_Alpes.html"
    url = "https://www.tripadvisor.fr/Restaurant_Review-g187265-d15114321-Reviews-L_affreux_Jojo-Lyon_Rhone_Auvergne_Rhone_Alpes.html"
    scraper = TripadvisorScraper(url)
    scraper.scrapper()
    data = scraper.data
    print(data)

if __name__ == "__main__":
    main()

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Pas de bannière cookies trouvée.


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Pas de bannière cookies trouvée.


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Pas de bannière cookies trouvée.


In [1]:
import time
import math
import re
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import undetected_chromedriver as uc
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [2]:
# Fonction pour extraire les nombres et calculer les pages
def extraire_infos(texte):
    """
    Extrait le nombre de commentaires par page, le nombre total de commentaires,
    et calcule le nombre de pages à partir d'un texte donné.
    """
    # Nettoyer le texte pour supprimer les espaces insécables
    texte = texte.replace("\u202f", "")  # Remplace les espaces insécables par rien

    # Extraire les chiffres du texte
    chiffres = [int(s) for s in re.findall(r'\d+', texte)]
    
    if len(chiffres) >= 2:
        nb_commentaires_par_page = chiffres[1]  # Exemple : "15" (2e chiffre)
        nb_total_commentaires = chiffres[-1]   # Exemple : "1300" (dernier chiffre)
        nb_pages = math.ceil(nb_total_commentaires / nb_commentaires_par_page)
        return nb_commentaires_par_page, nb_total_commentaires, nb_pages
    else:
        return None, None, None

  
def scraper_infos_restaurant(driver):
    """
    Scrape les informations globales sur le restaurant.
    """
    nom = driver.find_element(By.XPATH, "//h1[@class='biGQs _P egaXP rRtyp']").text 
    adresse = driver.find_element(By.XPATH, "//div[contains(text(), 'Emplacement et coordonnées')]/following::span[contains(@class, 'biGQs _P pZUbB hmDzD')][1]").text 
    note_globale = re.search(r"(\d+,\d+)", driver.find_elements(By.XPATH, "//div[@class='biGQs _P vvmrG']")[0].text).group(1)
    note_globale = float(note_globale.replace(',', '.'))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span//button[@class='ypcsE _S wSSLS']"))).click()
    horaires = [
        f"{lines[0]} : {' - '.join(lines[1:])}"
        for e in driver.find_elements("xpath", "//div[@class='VFyGJ Pi']")
        if len(lines := e.text.splitlines()) >= 2
    ]

    time.sleep(3)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span//button[@class='ypcsE _S wSSLS']"))).click()

    # Localiser les éléments des notes
    notes = driver.find_elements(By.XPATH, "//div[@class='khxWm f e Q3']/div/div")
    # Extraire les notes pour chaque catégorie à partir de l'innerHTML
    note_cuisine = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[1].get_attribute("innerHTML")).group(1)
    note_service = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[3].get_attribute("innerHTML")).group(1)
    note_rapportqualiteprix = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[5].get_attribute("innerHTML")).group(1)
    note_ambiance = re.search(r'<title[^>]*>([\d.,]+) sur [\d.,]+', notes[7].get_attribute("innerHTML")).group(1)

    # Conversion des notes en float avec des points
    note_cuisine = float(note_cuisine.replace(',', '.'))
    note_service = float(note_service.replace(',', '.'))
    note_rapportqualiteprix = float(note_rapportqualiteprix.replace(',', '.'))
    note_ambiance = float(note_ambiance.replace(',', '.'))

    classement = int((re.search(r'\d+', driver.find_element(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB hmDzD')]//b/span").text.replace("\u202f", "").replace(" ", "")).group()))


    #click pour avoir détails
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[class='UikNM _G B- _S _W _T c G_ wSSLS ACvVd']"))).click()
    time.sleep(2)
    #infos pratiques
    try:
        infos_pratiques = driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'Infos pratiques')]]/following-sibling::div[contains(@class, 'biGQs')]").text.strip()
    except Exception:
        infos_pratiques = "Non renseigné"

    #fourchette de prix
    try:
        fourchette_prix = driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'FOURCHETTE DE PRIX')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD')]").text.strip().replace("€", "").replace("\xa0", "")
    except Exception:
        fourchette_prix = "Non renseigné"  # Valeur par défaut
    #type cuisines
    try:
        types_cuisines = [item.strip() for item in driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'CUISINES')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD')]").text.strip().split(",")]
    except Exception:
        types_cuisines = "Non renseigné"
    #regimes
    try:
        regimes = [item.strip() for item in driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'Régimes spéciaux')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU W hmDzD')]").text.strip().split(",")]
    except Exception:
        regimes = "Non renseigné"
    #repas
    try:
        repas = [item.strip() for item in driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'Repas')]]/following-sibling::div[contains(@class, 'biGQs _P pZUbB alXOW eWlDX GzNcM ATzgx UTQMg TwpTY hmDzD')]").text.strip().split(",")]
    except Exception:
        repas = "Non renseigné"
    #fonctionnalités
    try:
        fonctionnalites = [item.strip() for item in driver.find_element(By.XPATH, "//div[contains(@class, 'Wf') and ./div[contains(text(), 'FONCTIONNALITÉS')]]/following-sibling::div[contains(@class, 'biGQs')]").text.strip().split(",")]
    except Exception:
        fonctionnalites = "Non renseigné"
    
    time.sleep(5)
    driver.find_element(By.XPATH, "//button[@aria-label='Fermer']").click()
    time.sleep(2)

    #quartier = driver.find_elements(By.XPATH,"//div[@class='akmhy e j']//a[@class='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS']/span[contains(@class, 'biGQs _P pZUbB hmDzD')]")
    #quartier = quartier[1].text
    
    try:
    # Récupérer l'élément `<a>` contenant le lien Google Maps
        google_maps_link = driver.find_element(By.XPATH,"//div[@class='akmhy e j']//a[@class='BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS']").get_attribute("href")
        # Extraire les coordonnées géographiques du lien
        if "@" in google_maps_link:
            coordinates = google_maps_link.split("@")[1].split(",")[:2]  # Prendre latitude et longitude
            latitude, longitude = float(coordinates[0]), float(coordinates[1])
            #print(f"Latitude : {latitude}, Longitude : {longitude}")
        else:
            latitude, longitude = "Non renseigné", "Non renseigné"
            #print("Coordonnées introuvables dans le lien.")
    except NoSuchElementException:
        latitude, longitude = "Non renseigné", "Non renseigné"
        #print("Lien Google Maps introuvable.")

    nb_avis = int((driver.find_element(By.XPATH, "//span[@class='GPKsO']").get_attribute("innerText").replace("\u202f", "").split()[0]))

    avis_list = {
        cat.text.strip(): int(nb.text.replace(" ", "").replace("\u202f", ""))
        for cat, nb in zip(
            driver.find_elements(By.XPATH, "//div[@class='biGQs _P fiohW hmDzD']/div[@class='Ygqck o W q']"),
            driver.find_elements(By.XPATH, "//div[@class='biGQs _P fiohW biKBZ osNWb']")
        )
    }
        # Créer des variables spécifiques
    nbExcellent = avis_list.get("Excellent", 0)
    nbTrèsBon = avis_list.get("Très bon", 0)
    nbMoyen = avis_list.get("Moyen", 0)
    nbMédiocre = avis_list.get("Médiocre", 0)
    nbHorrible = avis_list.get("Horrible", 0)

    
    return {
        "nom": nom, 
        "adresse": adresse, 
        "classement": classement, 
        "horaires": horaires,
        "note_globale": note_globale, 
        "note_cuisine":note_cuisine, 
        "note_service":note_service, 
        "note_rapportqualiteprix":note_rapportqualiteprix, 
        "note_ambiance":note_ambiance,
        "infos_pratiques":infos_pratiques,
        "repas":repas, 
        "regimes": regimes,
        "fonctionnalites":fonctionnalites,
        "fourchette_prix": fourchette_prix, 
        "types_cuisines": types_cuisines, 
        "latitude": latitude, 
        "longitude": longitude,
        "nb_avis": nb_avis,
        "nbExcellent": nbExcellent,
        "nbTrèsBon": nbTrèsBon,
        "nbMoyen": nbMoyen,
        "nbMédiocre": nbMédiocre,
        "nbHorrible": nbHorrible
        }

# Fonction pour scraper les avis d'une page
# def scraper_page(driver):
#     """
#     Récupère les avis d'une seule page.
#     """
#     data = []
#         # Récupération des éléments sur la page
#     pseudos = driver.find_elements(By.XPATH, "//span[@class='biGQs _P fiohW fOtGX']/a") 
#     titres = driver.find_elements(By.XPATH, "//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']")
#     etoiles = driver.find_elements(By.XPATH, "//div[@class='OSBmi J k']")
#     nb_etoiles = ([re.search(r'(\d+),', etoile.get_attribute("textContent")).group(1) for etoile in etoiles])
#     nb_etoiles = [int(etoile) for etoile in nb_etoiles]
#     dates = [re.search(r"\d{1,2}\s\w+\s\d{4}", elem.text.strip()).group(0) for elem in driver.find_elements(By.XPATH, "//div[contains(@class, 'biGQs _P pZUbB ncFvv osNWb')]")]
#     experiences = driver.find_elements(By.XPATH, "//span[@class='DlAxN']")
#     reviews = driver.find_elements(By.XPATH, "//div[@data-test-target='review-body']//span[@class='JguWG' and not(ancestor::div[contains(@class, 'csNQI')])]")
#     for i in range(len(titres)):
#         avis = {
#             "pseudo": pseudos[i].text if i < len(pseudos) else "",
#             "titre_review": titres[i].text if i < len(titres) else "",
#             "nb_etoiles": nb_etoiles[i] if i < len(nb_etoiles) else "",
#             "date": dates[i] if i < len(dates) else "",
#             "experience": experiences[i].text if i < len(experiences) else "",
#             "review": reviews[i].text if i < len(reviews) else ""
#         }
#         data.append(avis)
#     return data


def scraper_page(driver):
    """
    Récupère les avis d'une seule page, en filtrant ceux sans pseudo.
    """
    data = []

    # Récupération des conteneurs d'avis
    avis_containers = driver.find_elements(By.XPATH, "//div[@data-automation='reviewCard']")
    for container in avis_containers:
        try:
            # Récupération des éléments dans le conteneur d'avis
            pseudo_elem = container.find_element(By.XPATH, ".//span[@class='biGQs _P fiohW fOtGX']/a")
            titre_elem = container.find_element(By.XPATH, ".//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']")
            etoile_elem = container.find_element(By.XPATH, ".//div[@class='OSBmi J k']")
            date_elem = container.find_element(By.XPATH, ".//div[contains(@class, 'biGQs _P pZUbB ncFvv osNWb')]")
            experience_elem = container.find_element(By.XPATH, ".//span[@class='DlAxN']")
            review_elem = container.find_element(By.XPATH, ".//div[@data-test-target='review-body']//span[@class='JguWG' and not(ancestor::div[contains(@class, 'csNQI')])]")

            # Vérification de la présence du pseudo
            pseudo = pseudo_elem.text
            if not pseudo:
                continue  # Ignorer cet avis s'il n'y a pas de pseudo

            # Traitement des données
            nb_etoiles_match = re.search(r'(\d+),', etoile_elem.get_attribute("textContent"))
            nb_etoiles = int(nb_etoiles_match.group(1)) if nb_etoiles_match else None
            date_match = re.search(r"\d{1,2}\s\w+\s\d{4}", date_elem.text)
            date = date_match.group(0) if date_match else ""

            # Structure des données
            avis = {
                "pseudo": pseudo,
                "titre_review": titre_elem.text if titre_elem else "",
                "nb_etoiles": nb_etoiles,
                "date": date,
                "experience": experience_elem.text if experience_elem else "",
                "review": review_elem.text if review_elem else ""
            }
            data.append(avis)
        except Exception as e:
            # Gestion des erreurs sur un avis particulier
            #print(f"Erreur lors du traitement d'un avis : {e}")
            continue

    return data

        
# Fonction pour scraper les avis de toutes les pages
def scraper_toutes_pages(driver, nb_pages):
    """
    Scrape les avis de toutes les pages en utilisant la fonction `scraper_page`.
    """
    all_data = []
    actions = ActionChains(driver)
     
    for page in range(1, nb_pages + 1):
        print(f"Scraping de la page {page}...")
        time.sleep(5) 
        try:
            # Recharger les avis dynamiquement pour chaque page
            data = scraper_page(driver)
            print(f"Données collectées pour la page {page} : {len(data)} avis")
            all_data.extend(data)

            # Navigation vers la page suivante
            next_button = WebDriverWait(driver, 50).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@aria-label='Page suivante']"))
            )

            # Scroll et clic
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(5)
            actions.move_to_element(next_button).click().perform()
            print("Page suivante chargée.")
        
        except Exception as e:
            #print(f"Erreur rencontrée à la page {page} : {e}")
            break  # Arrêter la boucle, mais conserver les données collectées jusqu'ici

    return all_data

def test_scraping(driver, nbPages_texte):
    """
    Teste l'ensemble du processus de scraping :
    - Extraction d'informations globales sur le restaurant
    - Scraping des avis sur toutes les pages
    - Regroupement des données
    """
    avis = []  # Init
    infos_restaurant = {
        "nom": "Non disponible",
        "adresse": "Non disponible",
        "classement": "Non disponible",
        "horaires": [],
        "note_globale": "Non disponible",
        "note_cuisine": "Non disponible",
        "note_service": "Non disponible",
        "note_rapportqualiteprix": "Non disponible",
        "note_ambiance": "Non disponible",
        "repas": [],
        "infos_pratiques": "Non disponible",
        "regimes": [],
        "fonctionnalites": "Non disponible",
        "fourchette_prix": "Non disponible",
        "types_cuisines": [],
        "latitude" : "Non disponible",
        "longitude" : "Non disponible",
        "nb_avis": "Non disponible",
        "nbExcellent":"Non disponible",
        "nbTrèsBon": "Non disponible",
        "nbMoyen": "Non disponible",
        "nbMédiocre": "Non disponible",
        "nbHorrible": "Non disponible"

    }
    try:
        # Étape 1 : Extraire les infos globales
        infos_restaurant = scraper_infos_restaurant(driver)
        # print(f"Nom : {infos_restaurant['nom']}")
        # print(f"Adresse : {infos_restaurant['adresse']}")
        # print(f"Classement : {infos_restaurant['classement']}")
        # print(f"Horaires : {infos_restaurant['horaires']}")
        # print(f"Note globale : {infos_restaurant['note_globale']}")
        # print(f"Note cuisine : {infos_restaurant['note_cuisine']}")
        # print(f"Note service : {infos_restaurant['note_service']}")
        # print(f"Note rapport qualité prix : {infos_restaurant['note_rapportqualiteprix']}")
        # print(f"Note ambiance : {infos_restaurant['note_ambiance']}")
        # print(f"Infos pratiques : {infos_restaurant['infos_pratiques']}")
        # print(f"Repas : {infos_restaurant['repas']}")
        # print(f"Régimes : {infos_restaurant['regimes']}")
        # print(f"Fourchette de prix : {infos_restaurant['fourchette_prix']}")
        # print(f"Fonctionnalités : {infos_restaurant['fonctionnalites']}")
        # print(f"Type de cuisine : {infos_restaurant['types_cuisines']}")
        # print(f"Latitude : {infos_restaurant['latitude']}")
        # print(f"Longitude : {infos_restaurant['longitude']}")
        # print(f"NbAvis : {infos_restaurant['nb_avis']}")
        # print(f"Nb avis Excellent : {infos_restaurant['nbExcellent']}")
        # print(f"Nb avis Très bon : {infos_restaurant['nbTrèsBon']}")
        # print(f"Nb avis Moyen : {infos_restaurant['nbMoyen']}")
        # print(f"Nb avis Médiocre : {infos_restaurant['nbMédiocre']}")        
        # print(f"Nb avis Horrible : {infos_restaurant['nbHorrible']}")

        # Étape 2 : Extraire les infos pour les pages d'avis
        nb_commentaires_par_page, nb_total_commentaires, nb_pages = extraire_infos(nbPages_texte)
        print(f"Nombre de pages : {nb_pages}")

        # **Estimation du temps total** :
        average_time_per_page = 15  # Temps moyen par page en secondes
        estimated_total_time = average_time_per_page * nb_pages
        # Arrondir en minutes
        estimated_total_time_minutes = math.ceil(estimated_total_time / 60)
        print(f"Temps estimé pour terminer le scraping : {estimated_total_time_minutes} minutes.\n")

        # Étape 3 : Scraper les avis
        avis = scraper_toutes_pages(driver, nb_pages)
        print(f"Scraping terminé. Total d'avis collectés : {len(avis)}")

    except Exception as e:
        print(f"Erreur générale : {e}")

    # Étape 4 : Regrouper les données, même partielles
    restaurant_data = {
        "nom": infos_restaurant["nom"],
        "adresse": infos_restaurant["adresse"],
        "classement": infos_restaurant["classement"],
        "horaires": infos_restaurant["horaires"],
        "note_globale": infos_restaurant["note_globale"],
        "note_cuisine": infos_restaurant["note_cuisine"],
        "note_service": infos_restaurant["note_service"],
        "note_rapportqualiteprix": infos_restaurant["note_rapportqualiteprix"],
        "note_ambiance": infos_restaurant["note_ambiance"],
        "infos_pratiques": infos_restaurant["infos_pratiques"],
        "repas": infos_restaurant["repas"],
        "regimes": infos_restaurant["regimes"],
        "fourchette_prix": infos_restaurant["fourchette_prix"],
        "fonctionnalités": infos_restaurant["fonctionnalites"],
        "type_cuisines": infos_restaurant["types_cuisines"],
        "latitude": infos_restaurant["latitude"],
        "longitude": infos_restaurant["longitude"],
        "nb_avis": infos_restaurant["nb_avis"],
        "nbExcellent": infos_restaurant["nbExcellent"],
        "nbTrèsBon": infos_restaurant["nbTrèsBon"],
        "nbMoyen": infos_restaurant["nbMoyen"],
        "nbMédiocre": infos_restaurant["nbMédiocre"],
        "nbHorrible": infos_restaurant["nbHorrible"],
        "avis": avis  # Liste des avis
    
    }

    return restaurant_data

In [56]:
# Fonction pour configurer le driver
def create_driver():
    service = Service('C:/Users/Ihnhn/Desktop/M2 SISE/NLP/Projet/chromedriver.exe')
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    ]
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--incognito")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    user_agent = random.choice(user_agents)
    options.add_argument(f'--user-agent={user_agent}')

    return uc.Chrome(options=options, service=service)

# URL à visiter
#changer avec l'URL du restaurant à scraper
url = "https://www.tripadvisor.fr/Restaurant_Review-g187265-d4059959-Reviews-Mama_Restaurant_Lyon-Lyon_Rhone_Auvergne_Rhone_Alpes.html"

# Boucle pour redémarrer le navigateur jusqu'à ce que l'élément soit trouvé
found = False
attempts = 0
max_attempts = 20

while not found and attempts < max_attempts:
    driver = create_driver()  # Créer un nouveau navigateur
    try:
        driver.get(url)
        time.sleep(3)
        
        # Rendre Selenium indétectable
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        time.sleep(3)
        # Accepter les cookies
        try:
            WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-accept-btn-handler']"))
            ).click()
        except TimeoutException:
            print("Pas de bannière cookies trouvée.")

        # Chercher le nom
        nom = driver.find_element(By.XPATH, "//h1[@class='biGQs _P egaXP rRtyp']")
        print(f"Restaurant : {nom.text}")
        found = True  # Si le nom est trouvé, sortir de la boucle
    except NoSuchElementException:
        print(f"Nom non trouvé, tentative {attempts + 1}/{max_attempts}. Redémarrage...")
        attempts += 1
        driver.quit()  # Fermer le navigateur avant de recommencer
        time.sleep(10)  # Attendre avant de redémarrer un nouveau navigateur
if not found:
    print("Échec : le nom n'a pas été trouvé après plusieurs tentatives.")
    time.sleep(2)
    driver.quit()  # Fermer le dernier navigateur si le nom n'est pas trouvé
else:
    print("La page a été chargée avec succès.")
# Le driver reste ouvert si le nom a été trouvé


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Nom non trouvé, tentative 1/20. Redémarrage...


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Nom non trouvé, tentative 2/20. Redémarrage...


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Restaurant : Mama Restaurant Lyon
La page a été chargée avec succès.


In [ ]:
#code pour executer le scraping
#On recupère dabord le nb de pages à scraper et on appelle la fonction de scraping puis ferme le driver
nbPages_texte = driver.find_element("xpath", "//div[@class='Ci']").text
data = test_scraping(driver, nbPages_texte)
driver.quit()

Nom : Mama Restaurant Lyon
Adresse : 13 Rue Domer, 69007 Lyon France
Classement : 218
Horaires : ['dim : 19:00-23:00', 'lun : 12:00-15:00 - 19:00-23:00', 'mar : 12:00-15:00 - 19:00-23:00', 'mer : 12:00-15:00 - 19:00-23:00', 'jeu : 12:00-15:00 - 19:00-0:00', 'ven : 12:00-15:00 - 19:00-0:00', 'sam : 12:00-15:00 - 19:00-0:00']
Note globale : 4.0
Note cuisine : 3.8
Note service : 4.1
Note rapport qualité prix : 3.4
Note ambiance : 4.2
Infos pratiques : À deux pas de la place Jean Macé et de son métro qui vous emmène vers la gare Part-Dieu, le Mama Shelter est posé sur un ilot à trois rues, tel un triangle. Le Mama Shelter Lyon est un lieu de vie où le restaurant va jouer son rôle de mixité en mélangeant les profils et les plats, veillant à ce que la capitale de la gastronomie puisse s’honorer de ce récif. Le bar et ses tables d’hôtes vont vivre sous les vibrations de la scène « live » qui prouvent que Lyon est aussi une ville d’hospitalité et de convivialité.
Repas : ['Petit déjeuner', 'Dé

In [58]:
data

{'nom': 'Mama Restaurant Lyon',
 'adresse': '13 Rue Domer, 69007 Lyon France',
 'classement': 218,
 'horaires': ['dim : 19:00-23:00',
  'lun : 12:00-15:00 - 19:00-23:00',
  'mar : 12:00-15:00 - 19:00-23:00',
  'mer : 12:00-15:00 - 19:00-23:00',
  'jeu : 12:00-15:00 - 19:00-0:00',
  'ven : 12:00-15:00 - 19:00-0:00',
  'sam : 12:00-15:00 - 19:00-0:00'],
 'note_globale': 4.0,
 'note_cuisine': 3.8,
 'note_service': 4.1,
 'note_rapportqualiteprix': 3.4,
 'note_ambiance': 4.2,
 'infos_pratiques': 'À deux pas de la place Jean Macé et de son métro qui vous emmène vers la gare Part-Dieu, le Mama Shelter est posé sur un ilot à trois rues, tel un triangle. Le Mama Shelter Lyon est un lieu de vie où le restaurant va jouer son rôle de mixité en mélangeant les profils et les plats, veillant à ce que la capitale de la gastronomie puisse s’honorer de ce récif. Le bar et ses tables d’hôtes vont vivre sous les vibrations de la scène « live » qui prouvent que Lyon est aussi une ville d’hospitalité et de 

In [59]:
import json
# Exporter les données en JSON
with open("nom_restau.json", "w", encoding="utf-8") as f: json.dump(data, f, ensure_ascii=False, indent=4)
